# Remote monitoring for reforested plots

In [2]:
import ee
import geopandas as geopd
import numpy as np
import geemap
from ipyleaflet import GeoData
import os
os.environ['GDAL_DATA'] = os.environ['CONDA_PREFIX'] + r'\Library\share\gdal'
os.environ['PROJ_LIB'] = os.environ['CONDA_PREFIX'] + r'\Library\share'

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [4]:
# Load the park limits
roi = geopd.read_file("../dat/friches/roi.shp")

In [5]:
# get the centroid for centering the map
centroid_equal_area = roi.to_crs('+proj=cea').centroid.to_crs(roi.crs)
# Convert to geodata to add the layer to the map
roigd = GeoData(geo_dataframe = roi, style = {'color':'blue', 'fillOpacity':0.0})
# convert to earth engine compatible feature to clip the collections
g = [i for i in roi.geometry]
features=[]
for i in range(len(g)):
    g = [i for i in roi.geometry]
    x,y = g[i].exterior.coords.xy
    cords = np.dstack((x,y)).tolist()
    g=ee.Geometry.Polygon(cords)
    feature = ee.Feature(g)
    features.append(feature)
    roi = ee.FeatureCollection(features)

In [6]:
m = geemap.Map(center=[45.5, -73.5], zoom=9)
#m.add_basemap('Esri.WorldImagery')
#m.add_layer(roigd)
m

Map(center=[45.5, -73.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [13]:
#name of bands
inBands = ee.List(['QA60','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']);
outBands = ee.List(['QA60','blue','green','red','re1','re2','re3','nir','re4','waterVapor','cirrus','swir1','swir2']);
 
CloudCoverMax = 20

#function to get the data
def importData(studyArea,startDate,endDate):
 
    # Get Sentinel-2 data
    s2s =(ee.ImageCollection('COPERNICUS/S2')
          .filterDate(startDate,endDate)
          .filterBounds(studyArea)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',CloudCoverMax))
          .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',CloudCoverMax)))
    
    #sentinel bands are in scale of 0.0001
    def scaleBands(img):
        prop = img.toDictionary()
        t = (img.select(['QA60','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12'])
             .divide(10000))
        t = (t.addBands(img.select(['QA60'])).set(prop)
            .copyProperties(img,['system:time_start','system:footprint']))

        return ee.Image(t)
    
    
    s2s = s2s.map(scaleBands)
    s2s = s2s.select(inBands,outBands)
    
    return s2s

In [14]:
startyear = 2019
endyear = 2019

startDate = ee.Date.fromYMD(startyear,6,1)
endDate = ee.Date.fromYMD(endyear,8,31)

#gee assets to get the study area
studyArea = ee.FeatureCollection(roi)

print("Getting images") 
s2 = importData(studyArea, startDate, endDate)

s2 = s2.median().clip(studyArea)

Getting images


In [15]:
# get indexes
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['nir','red']).rename("ndvi")
    image = image.addBands(ndvi)

    return(image)

In [16]:
#get Indexes
print("getting indexes")
s2 = getNDVI(s2)

getting indexes


In [17]:
s2

In [12]:
visParams = { 'min': 0.0,
  'max': 8000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],}
Map.addLayer(s2, visParams,'NDVI')
#Map.add_layer(roi)
#Map.centerObject(roi, 12)
Map

AttributeError: 'Map' object has no attribute 'addLayer'